https://mccormickml.com/2020/03/10/question-answering-with-a-fine-tuned-BERT/

1. install transformer - to get pre-trained model

In [42]:
!pip install transformers

In [43]:
import torch

In [44]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


2. import BERT QA

In [45]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [46]:
question = "How quickly could COVID-19 vaccines stop the pandemic?"
answer_text = ''' The impact of COVID-19 vaccines on the pandemic will depend on several factors. These include the effectiveness of the vaccines; how quickly they are approved, manufactured, and delivered; the possible development of other variants and how many people get vaccinated
Whilst trials have shown several COVID-19 vaccines to have high levels of efficacy, like all other vaccines, COVID-19 vaccines will not be 100% effective. WHO is working to help ensure that approved vaccines are as effective as possible, so they can have the greatest impact on the pandemic. '''
            
encoding = tokenizer.encode_plus(text=question,text_pair=paragraph, add_special=True)

inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.


In [47]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 131 tokens.


In [48]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

[CLS]           101
how           2,129
quickly       2,855
could         2,071
co            2,522
##vid        17,258
-             1,011
19            2,539
vaccines     28,896
stop          2,644
the           1,996
pan           6,090
##de          3,207
##mic         7,712
?             1,029

[SEP]           102

the           1,996
impact        4,254
of            1,997
co            2,522
##vid        17,258
-             1,011
19            2,539
vaccines     28,896
on            2,006
the           1,996
pan           6,090
##de          3,207
##mic         7,712
will          2,097
depend       12,530
on            2,006
several       2,195
factors       5,876
.             1,012
these         2,122
include       2,421
the           1,996
effectiveness 12,353
of            1,997
the           1,996
vaccines     28,896
;             1,025
how           2,129
quickly       2,855
they          2,027
are           2,024
approved      4,844
,             1,010
manufactured  7,6

In [49]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [50]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]),return_dict=False ) # The segment IDs to differentiate question from answer_text
                                 

In [51]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "how quickly they are approved , manufactured , and delivered"


In [55]:
corrected_answer = ''

for word in answer.split():
    
    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word
print('Question = How quickly could COVID-19 vaccines stop the pandemic?')
print('Corrected Answer:'+corrected_answer)

Question = How quickly could COVID-19 vaccines stop the pandemic?
Corrected Answer: how quickly they are approved , manufactured , and delivered
